In [4]:
fun = lambda x: x ** 2

In [10]:
import numdifftools as nd

first = nd.Derivative(fun)
print first(3)

second = nd.Derivative(fun, n = 2)
print second(3)

third = nd.Derivative(fun, n = 3)
print third(3)

6.0
2.0
-1.05435936153e-13


In [15]:
from scipy.misc import derivative

print derivative(fun, 3, n = 1)
print derivative(fun, 3, n = 2)
# print derivative(fun, 3, n = 3)

6.0
2.0


In [119]:
from scipy.misc import derivative
import numpy as np

fun = lambda x, y: (x + y) ** 2
grad = lambda x: np.array([partial_derivative(fun, i, np.array(x)) for i in xrange(len(x))])

def partial_derivative(func, ix, point):
    args = point[:]
    def wraps(x):
        args[ix] = x
        return func(*args)
    return derivative(wraps, point[ix], dx = 1e-6)

def hess(grad, x):
    x = np.array(x).astype(float)
    hess = np.zeros_like(x)
    it = np.nditer(x, flags = ['multi_index'])
        
    while not it.finished:
        ix = it.multi_index
        hess[ix] = partial_derivative(grad, ix, hess)
        it.iternext()
        
    return hess

print grad([0, 0])
print hess(grad, [3, 1])

[ 0.  0.]


TypeError: <lambda>() takes exactly 1 argument (2 given)

In [21]:
foo = lambda x, y: x ** 2 + y ** 3

from scipy.misc import derivative
derivative(foo, 1, dx = 1e-6, args = (3, ))

def partial_derivative(func, var=0, point=[]):
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = 1e-6)

partial_derivative(foo, 1, [3, 1])

2.9999999995311555

In [52]:
x = np.array([[1, 2], [3, 4]])

it = np.nditer(x, flags = ['multi_index'])

while not it.finished:
    ix = it.multi_index
    print ix
    x[ix] = 2
    it.iternext()

(0, 0)
(0, 1)
(1, 0)
(1, 1)
